In [1]:
from pyspark.sql import SparkSession
import numpy as np # linear algebra
import pandas as pd # data processing , reading the csv file (pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

In [2]:
%matplotlib inline
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
          .master("local[*]") \
          .appName("Credit Card Analysis") \
          .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

22/12/26 09:08:47 WARN Utils: Your hostname, Abhisheks-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
22/12/26 09:08:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/26 09:08:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/26 09:08:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark

In [4]:
trxns = spark\
        .read\
        .options(header="true",inferSchema='True')\
        .csv("/Users/abhi011097/Dataset_Analysis/1_Credit_Card_Analysis/Dataset/Credit_card_transactions.csv")

trxns.registerTempTable("transactions")

In [5]:
trxns.printSchema()

root
 |-- index: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Card Type: string (nullable = true)
 |-- Exp Type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Amount: integer (nullable = true)



In [35]:
type(trxns)

pyspark.sql.dataframe.DataFrame

In [36]:
trxns_pd=trxns.toPandas()

In [39]:
type(trxns_pd)

pandas.core.frame.DataFrame

#### Describe Dataframe

In [6]:
trxns.describe().show()

+-------+-----------------+----------------+--------+---------+--------+------+------------------+
|summary|            index|            City|    Date|Card Type|Exp Type|Gender|            Amount|
+-------+-----------------+----------------+--------+---------+--------+------+------------------+
|  count|            26052|           26052|   26052|    26052|   26052| 26052|             26052|
|   mean|          13025.5|            null|    null|     null|    null|  null| 156411.5374251497|
| stddev|7520.708942646298|            null|    null|     null|    null|  null|103063.25428673124|
|    min|                0| Achalpur, India|1-Apr-14|     Gold|   Bills|     F|              1005|
|    max|            26051|Zunheboto, India|9-Sep-14|   Silver|  Travel|     M|            998077|
+-------+-----------------+----------------+--------+---------+--------+------+------------------+



In [41]:
trxns_pd.describe()

,index,Amount
count,26052.000000,26052.000000
mean,13025.500000,156411.537425
std,7520.708943,103063.254287
min,0.000000,1005.000000
25%,6512.750000,77120.250000
50%,13025.500000,153106.500000
75%,19538.250000,228050.000000
max,26051.000000,998077.000000


In [42]:
trxns_pd.describe(include='O')

,City,Date,Card Type,Exp Type,Gender
count,26052,26052,26052,26052,26052
unique,986,600,4,6,2
top,"Bengaluru, India",20-Sep-14,Silver,Food,F
freq,3552,65,6840,5463,13680


#### See first 2 rows 

In [7]:
trxns.show(2)

+-----+--------------------+---------+---------+--------+------+------+
|index|                City|     Date|Card Type|Exp Type|Gender|Amount|
+-----+--------------------+---------+---------+--------+------+------+
|    0|        Delhi, India|29-Oct-14|     Gold|   Bills|     F| 82475|
|    1|Greater Mumbai, I...|22-Aug-14| Platinum|   Bills|     F| 32555|
+-----+--------------------+---------+---------+--------+------+------+
only showing top 2 rows



In [8]:
spark.sql("Select * from transactions limit 2").show()

+-----+--------------------+---------+---------+--------+------+------+
|index|                City|     Date|Card Type|Exp Type|Gender|Amount|
+-----+--------------------+---------+---------+--------+------+------+
|    0|        Delhi, India|29-Oct-14|     Gold|   Bills|     F| 82475|
|    1|Greater Mumbai, I...|22-Aug-14| Platinum|   Bills|     F| 32555|
+-----+--------------------+---------+---------+--------+------+------+



#### Find total no of records

In [9]:
trxns.count()

26052

In [10]:
spark.sql("Select count(1) from transactions").show()

+--------+
|count(1)|
+--------+
|   26052|
+--------+



#### Find min max of amount

In [11]:
print("Amount Minimum Value -> " + str(trxns.agg({'Amount': 'min'}).collect()[0][0]))
print("Amount Maximum Value -> " + str(trxns.agg({'Amount': 'max'}).collect()[0][0]))
print("Mean Value -> " + str("{:.2f}".format(trxns.agg({'Amount': 'mean'}).collect()[0][0])))
#print("Amount Maximum Value -> " + trxns.groupby('Amount').median())

Amount Minimum Value -> 1005
Amount Maximum Value -> 998077
Mean Value -> 156411.54


In [24]:
spark.sql('''Select 
            min(Amount) as min_value,  
            max(Amount) as max_value,
            round(mean(Amount),2) as mean_value
          from transactions''').show()

+---------+---------+----------+
|min_value|max_value|mean_value|
+---------+---------+----------+
|     1005|   998077| 156411.54|
+---------+---------+----------+



#### Top 5 days having most transactions

In [65]:
spark.sql('''Select 
            Date,
            count(1) as cnt
          from transactions
          Group by Date
          order by cnt desc
          limit 5
          ''').show()

+---------+---+
|     Date|cnt|
+---------+---+
|20-Sep-14| 65|
|11-Aug-14| 61|
|15-Nov-14| 61|
|21-Dec-13| 61|
|20-Feb-15| 60|
+---------+---+



In [64]:
trxns\
    .groupby('Date')\
    .count()\
    .sort('count',ascending=False)\
    .limit(5)\
    .show()

+---------+-----+
|     Date|count|
+---------+-----+
|20-Sep-14|   65|
|11-Aug-14|   61|
|15-Nov-14|   61|
|21-Dec-13|   61|
|20-Feb-15|   60|
+---------+-----+



#### Top 5 citys having most transactions

In [68]:
spark.sql('''Select 
            city,
            count(1) as cnt
          from transactions
          Group by city
          order by cnt desc
          limit 5
          ''').show()

+--------------------+----+
|                city| cnt|
+--------------------+----+
|    Bengaluru, India|3552|
|Greater Mumbai, I...|3493|
|    Ahmedabad, India|3491|
|        Delhi, India|3482|
|    Hyderabad, India| 784|
+--------------------+----+



In [69]:
trxns\
    .groupby('city')\
    .count()\
    .sort('count',ascending=False)\
    .limit(5)\
    .show()

+--------------------+-----+
|                city|count|
+--------------------+-----+
|    Bengaluru, India| 3552|
|Greater Mumbai, I...| 3493|
|    Ahmedabad, India| 3491|
|        Delhi, India| 3482|
|    Hyderabad, India|  784|
+--------------------+-----+



#### Gender Distribution

In [79]:
spark.sql('''Select 
            `Gender`,
            count(1) as cnt
          from transactions
          Group by 1
          order by cnt desc
          ''').show()

+------+-----+
|Gender|  cnt|
+------+-----+
|     F|13680|
|     M|12372|
+------+-----+



In [80]:
trxns\
    .groupby('Gender')\
    .count()\
    .sort('count',ascending=False)\
    .show()

+------+-----+
|Gender|count|
+------+-----+
|     F|13680|
|     M|12372|
+------+-----+



#### Card Type used distribution highest to lowest 

In [73]:
spark.sql('''Select 
            `Card Type`,
            count(1) as cnt
          from transactions
          Group by 1
          order by cnt desc
          ''').show()

+---------+----+
|Card Type| cnt|
+---------+----+
|   Silver|6840|
|Signature|6447|
| Platinum|6398|
|     Gold|6367|
+---------+----+



In [75]:
trxns\
    .groupby('Card Type')\
    .count()\
    .sort('count',ascending=False)\
    .show()

+---------+-----+
|Card Type|count|
+---------+-----+
|   Silver| 6840|
|Signature| 6447|
| Platinum| 6398|
|     Gold| 6367|
+---------+-----+



#### Exp Type distribution highest to lowest 

In [76]:
spark.sql('''Select 
            `Exp Type`,
            count(1) as cnt
          from transactions
          Group by 1
          order by cnt desc
          ''').show()

+-------------+----+
|     Exp Type| cnt|
+-------------+----+
|         Food|5463|
|         Fuel|5257|
|        Bills|5078|
|Entertainment|4762|
|      Grocery|4754|
|       Travel| 738|
+-------------+----+



In [77]:
trxns\
    .groupby('Exp Type')\
    .count()\
    .sort('count',ascending=False)\
    .show()

+-------------+-----+
|     Exp Type|count|
+-------------+-----+
|         Food| 5463|
|         Fuel| 5257|
|        Bills| 5078|
|Entertainment| 4762|
|      Grocery| 4754|
|       Travel|  738|
+-------------+-----+



#### Stop Spark

In [81]:
spark.stop()